# Installation

In [ ]:
# remove `!` if running the line in a terminal
!pip install -U RelevanceAI[notebook]==2.0.0


# Setup

First, you need to set up a client object to interact with RelevanceAI.

In [ ]:
from relevanceai import Client

"""
You can sign up/login and find your credentials here: https://cloud.relevance.ai/sdk/api
Once you have signed up, click on the value under `Activation token` and paste it here
"""
client = Client()



# Data

You will need to have a dataset under your Relevance AI account. You can either use our e-commerce dataset as shown below or follow the tutorial on how to create your own dataset.

Our e-commerce dataset includes fields such as `product_title`, as well as the vectorized version of the field `product_title_clip_vector_`. Loading these documents can be done via:



## Load the data

In [ ]:
from relevanceai.utils.datasets import get_ecommerce_dataset_encoded

documents = get_ecommerce_dataset_encoded()
{k:v for k, v in documents[0].items() if '_vector_' not in k}


In [ ]:
for i, doc in enumerate(documents):
  # Creating price field in the number format to be used for metrics later
  try:
    doc["product_price"] = int(round(float((doc["product_price"].replace(" ",'').replace(",",'').replace("$",'')))))
  except Exception as e:
    del doc["product_price"]

documents[0].keys()


## Upload the data to Relevance AI

Run the following cell, to upload these documents into your personal Relevance AI account under the name `quickstart_clustering_aggregation`

In [5]:
ds = client.Dataset("quickstart_clustering_aggregation")
ds.insert_documents(documents)


while inserting, you can visit your dashboard at https://cloud.relevance.ai/dataset/quickstrt_clustering_kmeans/dashboard/monitor/


{'failed_documents': [], 'failed_documents_detailed': [], 'inserted': 739}

## Check the data

In [ ]:
ds.health()


# Clustering

We apply the Kmeams clustering algorithm to the vector field, `product_title_clip_vector_`, to perform clustering.

In [ ]:
from sklearn.cluster import KMeans

VECTOR_FIELD = "product_title_clip_vector_"
KMEAN_NUMBER_OF_CLUSTERS = 5
ALIAS = "kmeans_" + str(KMEAN_NUMBER_OF_CLUSTERS)

model = KMeans(n_clusters=KMEAN_NUMBER_OF_CLUSTERS)
clusterer = client.ClusterOps(alias=ALIAS, model=model)
clusterer.operate(dataset_id="quickstart_clustering_aggregation", vector_fields=["product_title_clip_vector_"])




Clustering results are automatically inserted into your datase. 
Here, we download a small sample and show the clustering results using our json_shower.

In [ ]:
## Here, we see how the first 5 data points are clustered:

from relevanceai import show_json

sample_documents = ds.sample(n=5)
samples = [{
    'product_title':d['product_title'],
    'cluster':d['_cluster_'][VECTOR_FIELD][ALIAS]
} for d in sample_documents]

show_json(samples, text_fields=['product_title', 'cluster'])



# Cluster aggregation

In [12]:
# Aggregate based on the clusters
result = clusterer.aggregate(
    dataset="quickstart_clustering_aggregation",
    vector_fields = [VECTOR_FIELD],
    groupby=[
      {"name": "query", "field": "query", "agg": "category"}
    ]
)


You can now visit the dashboard at https://cloud.relevance.ai/sdk/cluster/aggregation
